<a href="https://colab.research.google.com/github/Makstarr/CURE-algorithm/blob/main/CURE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Немного о данных

# Алгоритм CURE



In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.preprocessing import StandardScaler, normalize

In [2]:
lenght_data = 1

Алгоритм нельзя применять сразу к большой базе данных, ввиду большой сложности вычислений. Для того, чтобы применять алгоритм используют небольшую случайную выборку точек, для которой осуществляется разделение на n заданных кластеров. 
Затем в алгоритм вводится весь датасет и распределяется по полученным на первом этапе кластреам, что важно, за один проход.

### Первый шаг алгоритма
Для кластеризации выборки используется иерархическая кластеризация, которую можно описать следующим образом. 


---

```
 
*Провести кластеризацию для создания начальных кластеров *

  1) Присвоение точкам индексов и создание кластеров для каждой из точек

  2) Определение ближайших кластеров
      Для всех кластеров
        Берем найденную на предыдущем запуске цикла дистанцию (или очень большое число при первом запуске)
        Находим новую минимальную дистанцию
        Присваиваем найденный ближайший кластер тому, для которого вызвана функция

  3) Пока число кластеров больше заданного при запуске алгоритма
        3.1) Найти индекс кластера, который можно обьединить с соседом
                Для всех кластеров
                   Берем найденную на предыдущем запуске цикла дистанцию (или очень большое число при первом запуске)
                   Находим новую минимальную дистанцию

        3.2) Создаем новый кластер из двух соседей
        3.3) Пересчет связей в списке кластеров
              Для всех кластеров
                Дистанция до нового кластера
                Если найденная дистанция меньше самой близкой
                  Обьявляем кластер ближайшим соседом нового
                Если ближайший кластер текущего был одним из обьединенных
                  Если этот кластер ближе чем новая минималная дистанция
                    Присваиваем ему новый ближайший
                  Есои нет
                    То самый ближайший к нему новый, обьединенный
                Если не был
                  Если его ближайший сосед дальше чем новый кластер
                    То самый ближайший к нему новый, обьединенный
        3.4) Добавляем новый кластер в список

На выходе получаем n кластеров


```
---






Для каждого кластера необходимо найти точки-предтавители. Их число - параметр принимаемы на вход алгоритмом (rep_count). Для расчета точек представителей необходимо взять rep_count точек внутри каждого кластера на максимальном удалении друг от друга. Это необходимо для максимальной интерпретации начальной формы кластера. Первая точка выбирается случайно, остальные по расстоянию от нее.
Затем точки необходимо сдвинуть на заданнвый (alpha) процент расстояния в сторону центроиа кластера. Получаемые таким образом синтетические точки и будут представлять форму кластера в дальнейшем.

### Второй шаг алгоритма

Берется весь датасет, и сканируется каждая его точка. Каждая точка присваевается к самому ближайшему кластеру ( на основании расчета расстояния до всех его точек представителей )
```
Финальная кластеризация

  1) Словарь для n кластеров
  2) Для всех точек в массиве
        Подбираем к какому классу пренадлежит точка
          Для всех кластеров
            Минимальная дистанция = большое число
            Минимальное растояние от точки до кластера
            Если дистанция меньше чем минимальная
               Минимальная дистанция =  новая минимальная дистанция 
            Возвращаем индекс кластера для которого дистанция минимальна
        Присваиваем в словарь, в массив одного из классов 
  3) Возвращаем массивы индексов точек распределенные по классам
      

На выходе получаем кластеризованный набор данных.


```



Вот так просто можно описать работу алгоритма CURE.

Далее представлены методы кластеризации разделенные на методы кластера и методы иерархической кластеризации. Для всех действий в коде добавлены комментарии.

In [3]:
class Cluster(object):
    COUNTER = 0
    # Создание кластера на основе точек, их индексов и входных параметров
    def __init__(self, points, p_ids, rep_count, alpha):
        # Счетчик числа кластеров
        Cluster.COUNTER += 1
        self.id = Cluster.COUNTER
        # Добавление точе в формате массива
        self.points = np.array(points) if type(points) == type([]) else points
        # Число точек и число измерений
        self.n_points, self.dimensions = self.points.shape
        # Индексы точек
        self.p_ids = p_ids
        # Среднее значение по кластеру 
        self.mean = None
        # Параметры алгоритма
        self.alpha = alpha
        self.rep_count = rep_count
        # Считает точки-представители
        self.reps = self.assignReps()
        # Переменная ближайшего кластера
        self.closest = None
        # Переменная дистанции до ближайшего кластера
        self.closestDist = sys.maxsize

    #  Расчет центроида
    def getMean(self):
        # Если он уже посчитан возвращает значение
        if self.mean is not None:
            return self.mean
        # Если нет считает как сумма координат точек/число точек
        self.mean = self.points.sum(axis=0)/(self.n_points*1.0)
        return self.mean

    # Считает точки-представители
    def assignReps(self):
        # Если точек в кластере меньше чем точек-представителей
        # То все они и есть представители
        if self.n_points <= self.rep_count:
            return self.points[:,:]
        # Пустой сет и пустой массив
        tmp_set = set()
        reps = []
        # Для всех точек представителей
        for i in range(self.rep_count):
            max_dist = 0
            # Для всех точек в кластере (ищем самые далекие точки)
            for j in range(self.n_points):
                # Если точек представителей 0
                if i == 0:
                    # Расстояние между точкой и центроидом
                    min_dist = Cluster.p_distance(self.points[j], self.getMean())
                # Если точек представителей > 0
                else:
                    # Поиск минимального расстояния между точкой и массивом точек-представителей
                    min_dist = self.getClosestDist(self.points[j], reps)
                # Если расстояние больше максимального, новое максимальное
                if min_dist >= max_dist:
                    # максимальное расстояние
                    max_dist = min_dist
                    # индекс найденной точки 
                    max_point = j
            # если этой точки нет в списке
            if max_point not in tmp_set:
                # добавить
                tmp_set.add(max_point)
                # если есть массив представителей
                if reps is not None:
                    # Находим точку по индексу
                    point = self.points[max_point]
                    # Добавляем представителя сдвинутого на Alpha в сторону центроида 
                    reps.append(point + self.alpha * (self.getMean() - point))

                # если нет массива представителей
                else:
                    # Добавляем представителя сдвинутого на Alpha в сторону центроида 
                    point = self.points[max_point]
                    reps = [point + self.alpha * (self.getMean() - point)]
        # Возвращаем точки представители
        reps = np.array(reps)
        return reps
            
    # Поиск минимального расстояния между точкой и массивом точек
    def getClosestDist(self, point, points):
        points = np.array(points)
        k=np.empty(points.shape[0])
        for j in range(points.shape[0]):
            k[j]+=sum(((points[j]-point)**2)**0.5)

        return min(k)

    
    def __str__(self):
        return "[ID:%d] PointCount=%d, ClosestDist=%0.2f with [ID:%d]"%(self.id, self.n_points, self.closestDist, self.closest.id if self.closest else -1)
    
    def __repr(self):
        return self.__str__()
    


    @classmethod
    # Создаем новый кластер из двух соседей 
    def merge(cls, cluster1, cluster2):
        # Проверка что оба кластера в одном измерении
        if cluster1.dimensions != cluster2.dimensions:
            raise ValueError('Ошибка! Измерения точек не совпадают!')
        # Обьединяем точки кластеров в общий массив с сохранением
        combined_points = np.concatenate((cluster1.points, cluster2.points))
        # Обьединяем индексы точек кластеров в общий массив (Они не np.array поэтому просто +)
        combined_p_ids = cluster1.p_ids + cluster2.p_ids
        # Создаем новенький кластер
        new_cluster = Cluster(combined_points, combined_p_ids, cluster1.rep_count, cluster1.alpha)
        # Присваиваем ему среднее значение по кластеру
        new_cluster.mean = (cluster1.getMean()*cluster1.n_points + cluster2.getMean()*cluster2.n_points)/(new_cluster.n_points*1.0)
        return new_cluster
    
    # Минимальное растояние от точки до кластера
    def pointToClusterDistance(self, point):
        min_dist = sys.maxsize
        # Для всех точек-представителей
        for rep_index in range(len(self.reps)):
            # Вычисляется расстояние между точкой и точкой представителем
            dist = Cluster.p_distance(self.reps[rep_index], point)
            # Поиск минимального
            min_dist = min(min_dist, dist)
        return min_dist
    
    @staticmethod
    # Вычисляется расстояние между точкой и точкой представителем
    def p_distance(point1, point2):
        i=0
        for j in range(len(point1)):
          i+=((point1[j]-point2[j])**2)**0.5
        return ((1-(6*i**2)/lenght_data*(lenght_data**2-1))**2)**0.5

    @staticmethod
    # Расчет дистанции между текущим кластером и другим кластером
    def c_distance(cluster1, cluster2):
        min_dist = sys.maxsize
        # Для всех точек-представителей одного
        for rep_1_index in range(len(cluster1.reps)):
            # Для всех точек-представителей другого
            for rep_2_index in range(len(cluster2.reps)):
                #  Вычисляется расстояние между точками представителями
                dist = Cluster.p_distance(cluster1.reps[rep_1_index], cluster2.reps[rep_2_index])
                #  Берется минимальная
                min_dist = min(min_dist, dist)
        return min_dist

In [4]:
# Иерархическая кластеризация
class HeirarchicalClustering(object):
    # Создание обьекта метода, на основе входных параметров
    def __init__(self, cluster_count, rep_count, alpha):
        self.cluster_count = cluster_count
        self.clusters = []
        self.rep_count = rep_count
        self.alpha = alpha
    #
    #
    #
    # №1 ПЕРВНОНАЧАЛЬНАЯ КЛАСТЕРИЗАЦИЯ НА ОСНОВЕ МАЛЕНЬКОЙ ВЫБОРКИ
    #
    #
    #
    def fit(self, points):
        # Присвоение точкам индексов и создание кластеров для каждой из точек
        for p_index, point in enumerate(points):
            self.clusters.append(Cluster([point], [p_index], self.rep_count, self.alpha))
        
        # Определение ближайших кластеров
        for index in range(len(self.clusters)):
            self.assignClosestCluster(self.clusters[index])
          
        # Пока число кластеров больше заданного при запуске алгоритма
        while len(self.clusters) > self.cluster_count:

            # Найти индекс кластера, который можно обьединить с соседом
            u_index = self.getMergeCandidate()
            # Убрать кластер из массива кластеров и привоить переменной 
            # и привоить переменной u-кластер
            u_cluster = self.clusters.pop(u_index)
            # v-кластер равен соседу убранного
            v_cluster = u_cluster.closest
            # Найти ближайший и убрать его из массива
            self.removeCluster(v_cluster)
            # Создаем новый кластер из двух соседей (метод класса Cluster)
            new_cluster = Cluster.merge(u_cluster, v_cluster)
            # Пересчет связей в списке кластеров
            self.rebuildClosestClusterLinks(new_cluster, u_cluster.id, v_cluster.id)
            # Добавляем новый кластер в список
            self.clusters.append(new_cluster)

    # Определение ближайших кластеров
    def assignClosestCluster(self, cluster):
        # задание минимальной дистанции в ввиде очень большого int
        min_dist = sys.maxsize
        # переменная для ближайшего кластера
        closest = None
        # Для всех кластеров
        for i in range(len(self.clusters)):
            # Если индекс кластера тот же самый что и у текущего (уже обьедденины) ничего не делай
            if cluster.id == self.clusters[i].id: continue
            # Расчет дистанции между текущим кластером и кластером для которого вызвана функция (метод кластера)
            dist = Cluster.c_distance(cluster, self.clusters[i])
            # Если дистанция меньше минимальной (изначально огромного числа), значит это новая минимальная дистанция     
            if dist < min_dist:
                min_dist = dist
                # Значит текущий кластер самый близкий к тому, для которого вызвана функция
                closest = self.clusters[i]
        # Присваиваем найденный ближайший кластер тому, для которого вызвана функция
        cluster.closest = closest
        # Присваиваем дистанцию между ними
        cluster.closestDist = min_dist

    # Найти подходящие для обьединения кластеры
    def getMergeCandidate(self):
        # задание минимальной дистанции в ввиде очень большого int
        min_dist = sys.maxsize
        # Индекс подходящего для обьединения кластера
        cluster_index = 0
        # Для всех кластеров
        for i in range(len(self.clusters)):
            # Берем найденную на предыдущем шаге дистанцию до ближайшего кластера
            dist = self.clusters[i].closestDist
            # Ели дистанция меньше минимальной (изначально большое число)
            if dist < min_dist:
              # Новая минимальная дистанция между кластерами
              min_dist = dist
              cluster_index = i
        # Возвращаем айди кластера, который может быть объединен с соседом
        return cluster_index

    # Найти и убрать кластер из массива
    def removeCluster(self, cluster):
        # Для всех кластеров
        for i in range(len(self.clusters)):
            # Находим кластер в списке
            if self.clusters[i].id == cluster.id:
                # Убираем кластер из списка
                self.clusters[i] = self.clusters[-1]
                self.clusters.pop()
                return
        return "Кластер не был найден в списке"

    # Пересчет связей в списке кластеров
    def rebuildClosestClusterLinks(self, new_cluster, u_cluster_id, v_cluster_id):
        min_dist = sys.maxsize
        # Для всех кластеров
        for i in range(len(self.clusters)):
            # Дистанция между новым и текущим
            new_dist = Cluster.c_distance(new_cluster, self.clusters[i])
            # Если найденная дистанция меньше самой близкой
            # Обьявляем кластер ближайшим соседом нового
            if new_cluster.closestDist > new_dist:
                new_cluster.closestDist = new_dist
                new_cluster.closest = self.clusters[i]
            # Если ближайший кластер текущего был одним из обьединенных
            if self.clusters[i].closest.id in (u_cluster_id, v_cluster_id):
                # Если этот кластер ближе чем новая минималная дистанция
                # Присваиваем ему новый ближайший
                if self.clusters[i].closestDist < new_dist:
                    self.assignClosestCluster(self.clusters[i])
                # Есои нет
                # То самый ближайший к нему новый, обьединенный
                else:
                    self.clusters[i].closestDist = new_dist
                    self.clusters[i].closest = new_cluster
            # Если не был
            else:
              # Если его ближайший сосед дальше чем новый кластер
              # То самый ближайший к нему новый, обьединенный
                if self.clusters[i].closestDist > new_dist:
                    self.clusters[i].closestDist = new_dist
                    self.clusters[i].closest = new_cluster
    #
    #
    #
    # №2 КЛАСТЕРИЗАЦИЯ НА ОСНОВЕ ВСЕЙ ВЫБОРКИ
    #
    #
    #
    def predict(self, points):
        points = np.array(points)
        # Новый словарь, с пустым массивом, для каждого кластера 
        predictions = {cluster.id:[] for cluster in self.clusters}
        # Для всех точек в массиве
        for i in range(len(points)):
            # Подбираем к какому классу пренадлежит точка 
            label = self.predict_point(points[i])
            # Присваиваем в словарь, в массив одного из классов
            predictions[label].append(i)
        # Возвращаем массивы индексов точек распределенные по классам
        return predictions.values()

    #  Подбираем к какому классу пренадлежит точка 
    def predict_point(self, point):
        point = np.array(point)
        min_dist = sys.maxsize
        label = None
        # Для всех кластеров
        for i in range(len(self.clusters)):
            # Минимальное растояние от точки до кластера (Метод кластера)
            dist = self.clusters[i].pointToClusterDistance(point)
            # Если дистанция меньше чем минимальная, тогда относим к соответсвующему кластеру
            if dist < min_dist:
                min_dist = dist
                label = self.clusters[i].id
        # Возвращаем индекс кластера
        return label

In [5]:
cluster_count = 3

Комбинация числа точек представителей и коэфициента сжатия определяется подбором. (Т.к. данных о классах нет даже для небольшой выборки)
В качестве критерия правильности распределения взят общий рэйтинг счастья.

In [7]:
rep_count = 19
alpha = 0.1

In [9]:
df = pd.read_csv('data/Wholesale customers data.csv')
columns = list(df.columns)
df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867
438,1,3,10290,1981,2232,1038,168,2125


In [10]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled)
df_scaled

,0,1,2,3,4,5,6,7
0,1.448652,0.590668,0.052933,0.523568,-0.041115,-0.589367,-0.043569,-0.066339
1,1.448652,0.590668,-0.391302,0.544458,0.170318,-0.270136,0.086407,0.089151
2,1.448652,0.590668,-0.447029,0.408538,-0.028157,-0.137536,0.133232,2.243293
3,-0.690297,0.590668,0.100111,-0.624020,-0.392977,0.687144,-0.498588,0.093411
4,1.448652,0.590668,0.840239,-0.052396,-0.079356,0.173859,-0.231918,1.299347
...,...,...,...,...,...,...,...,...
435,-0.690297,0.590668,1.401312,0.848446,0.850760,2.075222,-0.566831,0.241091
436,-0.690297,0.590668,2.155293,-0.592142,-0.757165,0.296561,-0.585519,0.291501
437,1.448652,0.590668,0.200326,1.314671,2.348386,-0.543380,2.511218,0.121456
438,-0.690297,0.590668,-0.135384,-0.517536,-0.602514,-0.419441,-0.569770,0.213046


## Расчет

In [41]:
hc = HeirarchicalClustering(cluster_count, rep_count, alpha)
hc.fit(df_scaled.sample(100).values)
predictions = hc.predict(df_scaled.values)

In [42]:
centroids = [cluster.getMean() for cluster in hc.clusters]

In [43]:
centroids

[array([-0.69029709, -1.99534212, -0.79508735, -0.66539277, -0.28710225,
        -0.34829415, -0.60399652, -0.06988906]),
 array([ 1.44865163,  0.59066829, -0.79247513,  0.66382849,  1.21241984,
        -0.58544895,  1.82585376,  0.22298614]),
 array([-0.01369086, -0.04264038,  0.17264202, -0.00978063, -0.04866554,
         0.02026796, -0.09823124,  0.10773423])]

In [44]:
from sklearn.manifold import TSNE
import plotly.express as px

In [45]:
def compress(df, centroids):
    tsne = TSNE(n_components=2)
    concat_df = pd.concat([df, pd.DataFrame(centroids)])
    tnse_df = tsne.fit_transform(concat_df)

    return pd.DataFrame(tnse_df[:-len(centroids)]), pd.DataFrame(tnse_df[-len(centroids):])

In [46]:
tnse_df, tnse_centroids = compress(df_scaled, centroids)

C:\Anaconda\lib\site-packages\sklearn\manifold\_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

C:\Anaconda\lib\site-packages\sklearn\manifold\_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [47]:
fig = px.scatter(tnse_df, x=0, y=1)
fig.add_scatter(x=tnse_centroids[0], y=tnse_centroids[1])
fig.show()